<a href="https://colab.research.google.com/github/SylviaaaaW/StyleCLIP_study/blob/main/putback.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

/content/1にStyleCLIPから得る表情
/content/getに元の画像とshape_predictor_68_face_landmarks.dat
/content/outputに結果を保存
output.mp4,outut.zipを出力

In [14]:

import cv2
import dlib
import numpy

import shutil
import sys
import os

# !/usr/bin/python
# -*- coding: utf-8 -*-

PREDICTOR_PATH = "/content/get/shape_predictor_68_face_landmarks.dat"# モデル
SCALE_FACTOR = 1
FEATHER_AMOUNT = 11

FACE_POINTS = list(range(17, 68))
MOUTH_POINTS = list(range(48, 61))
RIGHT_BROW_POINTS = list(range(17, 22))
LEFT_BROW_POINTS = list(range(22, 27))
RIGHT_EYE_POINTS = list(range(36, 42))
LEFT_EYE_POINTS = list(range(42, 48))
NOSE_POINTS = list(range(27, 35))
JAW_POINTS = list(range(0, 17))

# Points used to line up the images.
ALIGN_POINTS = (LEFT_BROW_POINTS + RIGHT_EYE_POINTS + LEFT_EYE_POINTS +
                RIGHT_BROW_POINTS + NOSE_POINTS + MOUTH_POINTS)

# Points from the second image to overlay on the first. The convex hull of each
# element will be overlaid.
OVERLAY_POINTS = [
    LEFT_EYE_POINTS + RIGHT_EYE_POINTS + LEFT_BROW_POINTS + RIGHT_BROW_POINTS,
    NOSE_POINTS + MOUTH_POINTS,
]

# Amount of blur to use during colour correction, as a fraction of the
# pupillary distance.
COLOUR_CORRECT_BLUR_FRAC = 0.6

#dlibでランドマークを検出
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(PREDICTOR_PATH)


class TooManyFaces(Exception):
    pass


class NoFaces(Exception):
    pass


def get_landmarks(im):
    rects = detector(im, 1) #顔検出

    if len(rects) > 1:
        raise TooManyFaces #顔が複数
    if len(rects) == 0:
        raise NoFaces

    return numpy.matrix([[p.x, p.y] for p in predictor(im, rects[0]).parts()]) #顔の輪郭


def annotate_landmarks(im, landmarks):
    im = im.copy()
    for idx, point in enumerate(landmarks):
        pos = (point[0, 0], point[0, 1])
        cv2.putText(im, str(idx), pos,
                    fontFace=cv2.FONT_HERSHEY_SCRIPT_SIMPLEX,
                    fontScale=0.4,
                    color=(0, 0, 255))
        cv2.circle(im, pos, 3, color=(0, 255, 255))
    return im

# 図２ー＞図1
def draw_convex_hull(im, points, color):
    points = cv2.convexHull(points) # 凸包を探す
    cv2.fillConvexPoly(im, points, color=color) # 凸包を埋める


# get_face_mask()
def get_face_mask(im, landmarks):
    im = numpy.zeros(im.shape[:2], dtype=numpy.float64) # 定制数据类型

    for group in OVERLAY_POINTS:
        draw_convex_hull(im,
                         landmarks[group],
                         color=1)

    # array([[[0, 1, 2],
    #         [3, 4, 5]],
    #
    #        [[6, 7, 8],
    #         [9, 10, 11]]])
    #
    # In[61]: arr1.shape  # 
    # Out[61]: (2, 2, 3)  # 
    # In[62]: arr1.transpose((1, 0, 2))
    # Out[62]:
    # array([[[0, 1, 2],
    #         [6, 7, 8]],
    #
    #        [[3, 4, 5],
    #         [9, 10, 11]]])

    im = numpy.array([im, im, im]).transpose((1, 2, 0))

    im = (cv2.GaussianBlur(im, (FEATHER_AMOUNT, FEATHER_AMOUNT), 0) > 0) * 1.0
    im = cv2.GaussianBlur(im, (FEATHER_AMOUNT, FEATHER_AMOUNT), 0)

    return im

#Procrustes Analysi
def transformation_from_points(points1, points2): 
    '''
    Return an affine transformation [s * R | T] such that:
        sum ||s*R*p1,i + T - p2,i||^2
    is minimized.

    R：2x2 直交行列
    s：スカラー
    T：二次元ベクトル
    pi 、qi
    Ordinary Procrustes Analysis'''

    points1 = points1.astype(numpy.float64) # 
    points2 = points2.astype(numpy.float64)

    c1 = numpy.mean(points1, axis=0) 
    c2 = numpy.mean(points2, axis=0)
    points1 -= c1
    points2 -= c2

    s1 = numpy.std(points1) # 標準の差
    s2 = numpy.std(points2)
    points1 /= s1 #
    points2 /= s2

    U, S, Vt = numpy.linalg.svd(points1.T * points2) # singular value decomposition
    R = (U * Vt).T #

    return numpy.vstack([numpy.hstack(((s2 / s1) * R,
                                       c2.T - (s2 / s1) * R * c1.T)),
                         numpy.matrix([0., 0., 1.])])


def read_im_and_landmarks(fname): 
    # 図を読み込み
    # 関数cv2.imread()
    # cv2.IMREAD_COLOR：画像をカラー(RGB)で読込む　画像の透明度は無視される．デフォルト値
    # cv2.IMREAD_GRAYSCALE：グレースケール画像として読み込む
    #
    # import cv2
    # img = cv2.imread('lena.jpg', 0)
    
    im = cv2.imread(fname, cv2.IMREAD_COLOR)
    #im = cv2.imread(file_pathname+'/'+fname, cv2.IMREAD_COLOR)
    # import cv2
    # image = cv2.imread("D:/shape.bmp")
    # print(image.shape[0])
    # print(image.shape[1])
    # print(image.shape[2])
    # 結果
    # 300
    # 200
    # 3
    # shape.bmp　200ピクセル*300ピクセル　カラー
    im = cv2.resize(im, (im.shape[1] * SCALE_FACTOR,
                         im.shape[0] * SCALE_FACTOR))
    s = get_landmarks(im)

    return im, s


def warp_im(im, M, dshape):
    output_im = numpy.zeros(dshape, dtype=im.dtype)
    cv2.warpAffine(im,
                   M[:2],
                   (dshape[1], dshape[0]),
                   dst=output_im,
                   borderMode=cv2.BORDER_TRANSPARENT,
                   flags=cv2.WARP_INVERSE_MAP)
    return output_im

# 画像の肌色、照明の違いを修正
def correct_colours(im1, im2, landmarks1):
    # blur_amount
    blur_amount = COLOUR_CORRECT_BLUR_FRAC * numpy.linalg.norm(
        numpy.mean(landmarks1[LEFT_EYE_POINTS], axis=0) -
        numpy.mean(landmarks1[RIGHT_EYE_POINTS], axis=0))
    blur_amount = int(blur_amount)
    if blur_amount % 2 == 0:
        blur_amount += 1
    im1_blur = cv2.GaussianBlur(im1, (blur_amount, blur_amount), 0)
    im2_blur = cv2.GaussianBlur(im2, (blur_amount, blur_amount), 0)

    # 
    im2_blur += (128 * (im2_blur <= 1.0)).astype(im2_blur.dtype)

    return (im2.astype(numpy.float64) * im1_blur.astype(numpy.float64) /
            im2_blur.astype(numpy.float64))


#cnt = 123
for i in range(0,123):
  input_img= str(i).zfill(6)+".png"
  input_path="/content/1/"+input_img
  print(input_img)

  #sys.argvプログラム名称、背景図のpath、表情変化後の図のpath。
  sys.argv = ["SwapFace.py", "/content/get/000019.jpg", input_path]
  im1, landmarks1 = read_im_and_landmarks(sys.argv[1])
  im2, landmarks2 = read_im_and_landmarks(sys.argv[2])

  M = transformation_from_points(landmarks1[ALIGN_POINTS],
                                 landmarks2[ALIGN_POINTS])

  mask = get_face_mask(im2, landmarks2)
  warped_mask = warp_im(mask, M, im1.shape)
  combined_mask = numpy.max([get_face_mask(im1, landmarks1), warped_mask],
                            axis=0)

  warped_im2 = warp_im(im2, M, im1.shape)
  warped_corrected_im2 = correct_colours(im1, warped_im2, landmarks1)

  output_im = im1 * (1.0 - combined_mask) + warped_corrected_im2 * combined_mask

  #output_name=input_img +'.png'
  
  if os.path.exists('./output' + input_img + '.png'):
   os.remove('./output' + input_name + '.png')

  cv2.imwrite('/content/output/'+input_img, output_im)
  i+=1
 


#input_img="000017.png"
#input_path="/content/1/"+input_img
# sys.argvプログラム名称、背景図のpath、表情変化後の図のpath。
#sys.argv = ["SwapFace.py", "/content/get/195081.jpg", input_name]

#im1, landmarks1 = read_im_and_landmarks(sys.argv[1])
#im2, landmarks2 = read_im_and_landmarks(sys.argv[2])

#M = transformation_from_points(landmarks1[ALIGN_POINTS],
#                               landmarks2[ALIGN_POINTS])

#mask = get_face_mask(im2, landmarks2)
#warped_mask = warp_im(mask, M, im1.shape)
#combined_mask = numpy.max([get_face_mask(im1, landmarks1), warped_mask],
#                          axis=0)

#warped_im2 = warp_im(im2, M, im1.shape)
#warped_corrected_im2 = correct_colours(im1, warped_im2, landmarks1)

#output_im = im1 * (1.0 - combined_mask) + warped_corrected_im2 * combined_mask

#output_name=input_name +'.png'
#cv2.imwrite('/content/output'+input_img, output_im)
 




000000.png
000001.png
000002.png
000003.png
000004.png
000005.png
000006.png
000007.png
000008.png
000009.png
000010.png
000011.png
000012.png
000013.png
000014.png
000015.png
000016.png
000017.png
000018.png
000019.png
000020.png
000021.png
000022.png
000023.png
000024.png
000025.png
000026.png
000027.png
000028.png
000029.png
000030.png
000031.png
000032.png
000033.png
000034.png
000035.png
000036.png
000037.png
000038.png
000039.png
000040.png
000041.png
000042.png
000043.png
000044.png
000045.png
000046.png
000047.png
000048.png
000049.png
000050.png
000051.png
000052.png
000053.png
000054.png
000055.png
000056.png
000057.png
000058.png
000059.png
000060.png
000061.png
000062.png
000063.png
000064.png
000065.png
000066.png
000067.png
000068.png
000069.png
000070.png
000071.png
000072.png
000073.png
000074.png
000075.png
000076.png
000077.png
000078.png
000079.png
000080.png
000081.png
000082.png
000083.png
000084.png
000085.png
000086.png
000087.png
000088.png
000089.png
000090.png

In [15]:
!zip -r /content/output.zip /content/output
if os.path.exists('./output.mp4'):
   os.remove('./output.mp4')

! ffmpeg -r 30 -i output/%6d.png\
               -vcodec libx264 -pix_fmt yuv420p output.mp4


  adding: content/output/ (stored 0%)
  adding: content/output/000026.png (deflated 2%)
  adding: content/output/000100.png (deflated 2%)
  adding: content/output/000067.png (deflated 2%)
  adding: content/output/000058.png (deflated 2%)
  adding: content/output/000066.png (deflated 2%)
  adding: content/output/000076.png (deflated 2%)
  adding: content/output/000060.png (deflated 2%)
  adding: content/output/000095.png (deflated 2%)
  adding: content/output/000082.png (deflated 2%)
  adding: content/output/000079.png (deflated 2%)
  adding: content/output/000051.png (deflated 2%)
  adding: content/output/000040.png (deflated 2%)
  adding: content/output/000055.png (deflated 2%)
  adding: content/output/000005.png (deflated 2%)
  adding: content/output/000071.png (deflated 2%)
  adding: content/output/000043.png (deflated 2%)
  adding: content/output/000034.png (deflated 2%)
  adding: content/output/000074.png (deflated 2%)
  adding: content/output/000020.png (deflated 2%)
  adding: co